# Metaklasy - tworzenie interfejsów deklaratywnych
##### Hubert Bielenia
---

## SQLAlchemy - biblioteka dwóch interfejsów

In [ ]:
import sqlalchemy as sa
engine = sa.create_engine('sqlite:///:memory:')
metadata = sa.MetaData()

## Programowanie imperatywne

Opisuje przepływ sterowania (ang. *control flow*) za pomocą instrukcji warunkowych i wywołań funkcji.

In [ ]:
from sqlalchemy.orm import mapper

# Utwórz kolumny
c1 = sa.Column('id', sa.Integer, primary_key=True)
c2 = sa.Column('name', sa.Text)
c3 = sa.Column('salary', sa.Numeric(15, 6))

# Utwórz obiekt tabeli
person = sa.Table(
    'person', metadata, c1, c2, c3, extend_existing=True)

class Person(object):
    def __init__(self, name, salary):
        self.name = name
        self.salary = salary

# Zmapuj obiekt do klasy
mapper(Person, person)

## Programowanie deklaratywne
Za pomocą wyrażeń opisuje pożądany stan, nie zaś algorytm który ma do niego doprowadzić.

In [ ]:
del Person

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Deklarujemy mapping w jednym kroku!
class Person(Base):
    __tablename__ = 'person'
    
    id = sa.Column(sa.Integer, primary_key=True)
    name = sa.Column(sa.Text)
    salary = sa.Column(sa.Numeric(15, 6))

 * Paradygmaty programowania nie muszą się wykluczać - w większości popularnych języków można używać różnych, zależnie od problemów/potrzeb/preferencji.
 * W Pythonie (i większości języków obiektowych) utworzenie klasy jest przykładem **deklaracji**.
 * Do stworzenia deklaratywnego API potrzebujemy więc czegoś, co rozszerzy deklarację klasy o nową funkcjonalność.

### Metaklasa
To klasa, której instancją jest również klasa.

`type()` to nie funkcja do sprawdzania typów...

In [1]:
class Cls:
    pass

type.__class__ == Cls.__class__ # ...to klasa!

True

In [2]:
c = type('NewClass', (object,), {})
c.__class__ == Cls.__class__ # ...a nawet metaklasa!

True

In [3]:
object.__class__ == type

True

Zatem co należy zrobić, by napisać własną metaklasę?

In [4]:
import csv


class BaseModel(type):
  
  def __new__(cls, *args, **kwargs):
    new_cls = super().__new__(cls, *args, **kwargs)

    # Don't process the immediate subclass
    if new_cls.__name__ == 'Model':
      return new_cls

    # Create fields registry
    fields = {}
    for attrname in dir(new_cls):
      attr = getattr(new_cls, attrname)
      if issubclass(attr.__class__, BaseField):
        fields[attrname] = attr
    new_cls.fields = fields

    new_cls._filename = new_cls.__name__ + '.csv'
    open(new_cls._filename, 'a').close() ## Ensure that file exists

    # Load initial objects
    with open(new_cls._filename, 'r') as csvfile:
      reader = csv.DictReader(csvfile, fieldnames=fields.keys())
      new_cls.objects = [new_cls(**row) for row in reader]

    return new_cls

In [5]:
class Model(metaclass=BaseModel):

  def __init__(self, **kwargs):
    for k, v in kwargs.items():
      setattr(self, k, v)
  
  def __setattr__(self, name, value):
    if name in self.fields:
      value = self.fields[name](value).value
    return super().__setattr__(name, value)
  
  def __iter__(self):
    return iter(
      [(f, getattr(self, f)) for f in self.fields])
  
  def save(self):
    with open(self._filename, 'a') as csvfile:
      writer = csv.DictWriter(csvfile, fieldnames=self.fields.keys())
      writer.writerow(dict(self))
      if self not in self.objects:
        self.objects.append(self)

In [6]:
from decimal import Decimal


class BaseField:

  def __init__(self):
    self.value = self.data_type(False)
  
  def __call__(self, value):
    self.value = self.data_type(value)
    return self
  
  def __repr__(self):
    if hasattr(self, 'value'):
      return repr(self.value)
    else:
      return super().__repr__()

class DecimalField(BaseField):
  data_type = Decimal

class IntegerField(BaseField):
  data_type = int

class StringField(BaseField):
  data_type = str

In [7]:
class Person(Model):
  name = StringField()
  height_cm = IntegerField()
  salary = DecimalField()

In [8]:
with open('Person.csv', 'r') as f:
    print(f.read())

175,Jan Kowalski,1200.63



In [9]:
p = Person.objects[0]
dict(p)

{'height_cm': 175, 'name': 'Jan Kowalski', 'salary': Decimal('1200.63')}

In [10]:
p = Person(name='Maciej Nowak', height_cm=168, salary=3000)
p.save()

In [11]:
with open('Person.csv', 'r') as f:
    print(f.read())

175,Jan Kowalski,1200.63
168,Maciej Nowak,3000



**„Metaclasses are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don't.”**

*Tim Peters, Python core developer*

## Koniec!
### Więcej informacji:
https://realpython.com/python-metaclasses/
https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/